In [ ]:
from math import pi

def rocket(tstep=0.001, **kwargs):
    rnozzle = kwargs.get('rnozzle', 0.004) # радиус сопла, метры
    vbottle = kwargs.get('vbottle', 1.5) # объем бутылки, литры
    mrocket = kwargs.get('mrocket', 0.55) # масса пустой ракеты, кг
    pair = kwargs.get('pair', 6.0) # давление возд, атм
    patm = kwargs.get('patm', 1.0) # атмосферное давление, атм
    mwater = kwargs.get('mwater', 0.6) # масса воды, кг
    max_time = kwargs.get('max_time', 60) # время расчета, сек
    calc_air = kwargs.get('calc_air', False)
    v_crit = kwargs.get('v_crit', -2)
    b_d = kwargs.get('b_d', 0.0)
    
    sn = pi * rnozzle ** 2 # площадь сопла
    ro_water = 1000 # плотность воды, кг / м ** 3
    ro_air = 1.225 # плотность воздуха, кг / м ** 3
    g = 9.81
    ak = 1.4
    
    pstart = (pair + patm) * 100000
    avstart = vbottle / 1000 - mwater / ro_water
    
    #начальные знач
    tcur = 0.0 # время
    hcur = 0.0 # высота
    vcur = 0.0 # скорость
    pcur = pstart # давление воздуха, Па
    mvcur = mwater # масса воды
    avcur = avstart # обЪем воздуха, м ** 3
    
    t = []
    h = []
    h2 = []
    v = []
    v2 = []
    p = []
    p2 = []
    mw = []
    mw2 = []
    a = []
    a2 = []
    av = []
    thrust = []
    vc = []
    
    p_out = patm * 100000
    p_0 = (pair + patm) * 100000
    p_in = p_0
    v_0 = vbottle / 1000 - mwater / ro_water
    r_n = rnozzle
    m_0 = mwater
    m_t = m_0
    v_t = 0
    a_t = -g
    h_t = 0
    
    while tcur < max_time and hcur >= 0:
        t.append(tcur)
        h.append(hcur)
        h2.append(h_t)
        v.append(vcur)
        v2.append(v_t)
        p.append(pcur) # / 100000)
        p2.append(p_in)
        mw.append(mvcur)
        mw2.append(m_t)
        av.append(avcur)
        a2.append(a_t)
        vc.append(v_crit)

        if m_t > 0.0001:
            p_in = p_0 * ((v_0  + (m_0 - m_t) / ro_water) / v_0) ** -ak
            alpha = sn * ro_water * (2 * (p_in - p_out) / ro_water) ** 0.5
            f_a = 2 * sn * (p_in - p_out)
            
            m_t -= alpha * tstep
        else:
            if calc_air and p_in > p_out:
                p_in = p_0 * ((v_0  + (m_0 - m_t) / ro_air) / v_0) ** -ak
                alpha = sn * ro_air * (2 * (p_in - p_out) / ro_air) ** 0.5
                f_a = 2 * sn * (p_in - p_out)
            else:
                alpha = 0
                f_a = 0


        h_t += v_t * tstep
        v_t += a_t * tstep
        a_t = (f_a - b_d * v_t * abs(v_t)) / (mrocket + m_t) - g
        
        if mvcur > 0.0001:
            vex = (2 * (pcur - p_out) / ro_water) ** 0.5 # скорость выхлопа
            dmv = vex * ro_water * sn * tstep
            thrustcur = 2 * (pcur - p_out) * sn
        else:
            if calc_air and pcur > p_out:
                vex = (2 * (pcur - p_out) / ro_air) ** 0.5 # скорость выхлопа
                thrustcur = 2 * pcur * sn
            else:
                vex = 0
                thrustcur = 0
            dmv = vex * ro_air * sn * tstep
            
        thrust.append(thrustcur)
        # дельты
        dav = sn * vex * tstep
        # dp = (avcur / (avcur + dav)) ** ak
        
        #
        pcur = (pstart) * (avstart / avcur) ** ak
        avcur += dav
        mvcur -= dmv 
        
        #
        acur = thrustcur / (mrocket + mvcur)
        a.append(acur)
        hcur += vcur * tstep
        vcur += acur * tstep
        vcur -= g * tstep
        tcur += tstep

    return {'t':t,
           'h':h,
            'h2':h2,
           'v':v,
           'v2':v2,
            'vc':vc,
           'p':p, 
            'p2':p2,
           'mw':mw,
           'mw2':mw2,
           'a':a, 
            'a2':a2,
           'thrust':thrust,
           'av': av}
    
    

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

c1=['blue', 'green', 'purple', 'gray']
c2=['dark blue', 'dark green', 'deep purple']

#ts = [0.05, 0.01, 0.005, 0.001, 0.0005]
ts = [0.01]
p = [0.5, 0.6, 0.7, 0.75]
m = [0.55]
for i, k in enumerate(m):
    d = rocket(mwater=0.6, pair=6, max_time=12.7, patm=1, mrocket=k, calc_air=False, b_d=3.1415926 * 0.045 * 0.045 * 0.8)
    
    #if i == 0:
    #    ax2.plot(d['t'], d['vc'], color='red')
        
    #fig.add_trace(go.Scatter(x=d['t'], y=d['h'], name='Высота'))
    fig.add_trace(go.Scatter(x=d['t'], y=d['h2'], name='Высота'))
    #fig.add_trace(go.Scatter(x=d['t'], y=d['v'], name='Скорость'), secondary_y=True,)
    fig.add_trace(go.Scatter(x=d['t'], y=d['v2'], name='Скорость'), secondary_y=True,)
    #fig.add_trace(go.Scatter(x=d['t'], y=d['a'], name='Ускорение'), secondary_y=True,)

fig.update_xaxes(title_text="Время, <i>сек</i>")
fig.update_yaxes(title_text="Высота, <i>м</i>", secondary_y=False,)
fig.update_yaxes(title_text="Скорость, <i>м/с</i>", secondary_y=True,)

fig.show()

In [ ]:
import matplotlib.pyplot as plt

# https://cmdlinetips.com/2019/10/how-to-make-a-plot-with-two-different-y-axis-in-python-with-matplotlib/
# create figure and axis objects with subplots()
fig,ax = plt.subplots()

d = rocket(calc_air = False, max_time = 0.45)

ax.plot(d['t'], d['h'])
ax.set_xlabel('time, s')
ax.set_ylabel('height, m')

#twin object for two different y-axis on the sample plot
  
ax2 = ax.twinx()
ax2.plot(d['t'], d['v'], color='green')
ax2.set_ylabel('velocity, m/s', color='green')

ax2 = ax.twinx()
ax2.plot(d['t'], d['a'], color='purple')
ax2.set_ylabel('acc, m/(s*s)', color='purple')

plt.show()

In [ ]:
# https://cmdlinetips.com/2019/10/how-to-make-a-plot-with-two-different-y-axis-in-python-with-matplotlib/
# create figure and axis objects with subplots()
fig,ax = plt.subplots()

d = rocket(calc_air = False, max_time = 0.45)

ax.plot(d['t'], d['v'], color='green')
ax.set_xlabel('time, s')
ax.set_ylabel('velocity, m/s', color='green')

ax2 = ax.twinx()
ax2.plot(d['t'], d['a'], color='purple')
ax2.set_ylabel('acc, m/(s*s)', color='purple')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

mw = []
h = []

a = [0.3, 0.4, 0.5, 0.55, 0.6, 0.7, 0.75, 1.0, 1.4]
for i in a:
    d = rocket(mwater=i, )
    mw.append(i)
    h.append(max(d['h']))
    #h.append(d['p'][-1])
    i += 0.1
    
# https://cmdlinetips.com/2019/10/how-to-make-a-plot-with-two-different-y-axis-in-python-with-matplotlib/
# create figure and axis objects with subplots()
fig,ax = plt.subplots()

ax.plot(mw, h)
ax.set_xlabel('вода, кг')
ax.set_ylabel('высота, м')

plt.show()
